In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

30926


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数
30921,2021-10-30,47,沖縄県,11,50223,0,380,5.09
30922,2021-10-31,47,沖縄県,2,50225,3,383,4.34
30923,2021-11-01,47,沖縄県,1,50226,4,387,4.06
30924,2021-11-02,47,沖縄県,11,50237,4,391,3.99
30925,2021-11-03,47,沖縄県,6,50243,4,395,3.17


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数
42,静岡県,14476,26755,209,5137.96
43,香川県,24346,4700,38,3441.20
44,高知県,25662,4166,32,4175.50
45,鳥取県,20398,1668,5,2099.87
46,鹿児島県,30268,9102,66,3976.82


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数
30919,2021-10-28,47,沖縄県,9,50206,4,380,5.51,80.0
30920,2021-10-29,47,沖縄県,6,50212,0,380,4.96,72.0
30921,2021-10-30,47,沖縄県,11,50223,0,380,5.09,74.0
30922,2021-10-31,47,沖縄県,2,50225,3,383,4.34,63.0
30923,2021-11-01,47,沖縄県,1,50226,4,387,4.06,59.0
30924,2021-11-02,47,沖縄県,11,50237,4,391,3.99,58.0
30925,2021-11-03,47,沖縄県,6,50243,4,395,3.17,46.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
30916,2021-10-25,47,沖縄県,5,50167,0,372,7.43,108.0,-0.615385,-0.049736,-0.687500
30917,2021-10-26,47,沖縄県,12,50179,4,376,6.26,91.0,1.400000,0.034193,-0.586207
30918,2021-10-27,47,沖縄県,18,50197,0,376,6.13,89.0,0.500000,0.149956,-0.100000
30919,2021-10-28,47,沖縄県,9,50206,4,380,5.51,80.0,-0.500000,0.092814,-0.500000
30920,2021-10-29,47,沖縄県,6,50212,0,380,4.96,72.0,-0.333333,0.076941,-0.571429
30921,2021-10-30,47,沖縄県,11,50223,0,380,5.09,74.0,0.833333,0.247009,0.222222
30922,2021-10-31,47,沖縄県,2,50225,3,383,4.34,63.0,-0.818182,0.066633,-0.846154
30923,2021-11-01,47,沖縄県,1,50226,4,387,4.06,59.0,-0.500000,0.083117,-0.800000
30924,2021-11-02,47,沖縄県,11,50237,4,391,3.99,58.0,10.000000,1.311688,-0.083333
30925,2021-11-03,47,沖縄県,6,50243,4,395,3.17,46.0,-0.454545,1.175325,-0.666667


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
657,2021-11-03,北海道,13,0,62.0,-0.071429,0.485503,0.444444
1315,2021-11-03,青森県,0,0,3.0,-1.000000,NaN,-1.000000
1973,2021-11-03,岩手県,0,0,0.0,NaN,NaN,NaN
2631,2021-11-03,宮城県,1,0,11.0,-0.500000,NaN,inf
3289,2021-11-03,秋田県,0,0,8.0,NaN,NaN,-1.000000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,15792,14801,163,5812.48,2021-11-03,5,0,26.0,-0.285714,NaN,inf
1,京都府,17108,35892,291,9723.37,2021-11-03,5,0,33.0,0.666667,0.782653,4.000000
2,佐賀県,26978,5857,30,5027.77,2021-11-03,2,0,6.0,1.000000,NaN,0.000000
3,兵庫県,18424,78496,1395,10046.85,2021-11-03,25,0,111.0,1.272727,0.664254,0.388889
4,北海道,658,60779,1471,8099.82,2021-11-03,13,0,62.0,-0.071429,0.485503,0.444444


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,15792,14801,163,5812.48,2021-11-03,5,0,26.0,-0.285714,NaN,inf,1813859
1,京都府,17108,35892,291,9723.37,2021-11-03,5,0,33.0,0.666667,0.782653,4.000000,2545899
2,佐賀県,26978,5857,30,5027.77,2021-11-03,2,0,6.0,1.000000,NaN,0.000000,823810
3,兵庫県,18424,78496,1395,10046.85,2021-11-03,25,0,111.0,1.272727,0.664254,0.388889,5549568
4,北海道,658,60779,1471,8099.82,2021-11-03,13,0,62.0,-0.071429,0.485503,0.444444,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,658,60779,1471,8099.82,2021-11-03,13,0,62.0,-0.071429,0.485503,0.444444,5267762,115.379169,1153.791686,1.176970
41,青森県,1316,5895,38,3310.66,2021-11-03,0,0,3.0,-1.000000,NaN,-1.000000,1275783,46.206918,462.069176,0.235150
19,岩手県,1974,3486,53,1988.42,2021-11-03,0,0,0.0,NaN,NaN,NaN,1235517,28.214909,282.149092,0.000000
11,宮城県,2632,16269,118,4937.05,2021-11-03,1,0,11.0,-0.500000,NaN,inf,2292385,70.969754,709.697542,0.479850
36,秋田県,3290,1908,27,1381.00,2021-11-03,0,0,8.0,NaN,NaN,-1.000000,985416,19.362381,193.623810,0.811840
15,山形県,3948,3551,56,2304.30,2021-11-03,2,0,3.0,1.000000,NaN,1.000000,1082296,32.809878,328.098783,0.277188
35,福島県,4606,9483,175,3595.84,2021-11-03,0,0,1.0,NaN,NaN,NaN,1881981,50.388394,503.883939,0.053135
38,茨城県,5264,24405,218,5971.76,2021-11-03,1,0,17.0,0.000000,NaN,0.000000,2921436,83.537685,835.376849,0.581906
27,栃木県,5922,15448,116,5590.13,2021-11-03,2,0,8.0,inf,NaN,inf,1965516,78.595137,785.951374,0.407018
37,群馬県,6580,16804,176,6050.33,2021-11-03,11,0,47.0,1.750000,0.377381,0.100000,1969439,85.323790,853.237902,2.386466


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      36563.042553
std       67772.010355
min        1668.000000
25%        5343.000000
50%        9483.000000
75%       25580.000000
max      381707.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,8554,381707,3152,19190.76,2021-11-03,25,3,142.0,0.388889,0.062718,-0.305556,13834925,275.901026,2759.010258,1.026388
1,大阪府,17766,202562,3047,16087.13,2021-11-03,53,0,303.0,0.472222,0.486859,-0.196970,8849635,228.893056,2288.930560,3.423870
2,神奈川県,9212,168920,1308,12853.55,2021-11-03,7,0,63.0,-0.300000,-0.034184,-0.533333,9209442,183.420450,1834.204504,0.684081
3,埼玉県,7238,115623,1046,11009.45,2021-11-03,7,0,69.0,0.000000,0.065882,-0.500000,7390054,156.457585,1564.575847,0.933687
4,愛知県,15134,106677,1156,9884.13,2021-11-03,22,0,111.0,1.200000,0.171931,0.100000,7575530,140.817870,1408.178702,1.465244
5,千葉県,7896,100325,1025,11216.77,2021-11-03,13,0,65.0,-0.235294,0.654682,0.857143,6319772,158.747816,1587.478156,1.028518
6,兵庫県,18424,78496,1395,10046.85,2021-11-03,25,0,111.0,1.272727,0.664254,0.388889,5549568,141.445244,1414.452440,2.000156
7,福岡県,26320,74447,621,10207.06,2021-11-03,5,0,60.0,0.000000,0.056256,-0.444444,5129841,145.125356,1451.253557,1.169627
8,北海道,658,60779,1471,8099.82,2021-11-03,13,0,62.0,-0.071429,0.485503,0.444444,5267762,115.379169,1153.791686,1.176970
9,沖縄県,30926,50243,395,24196.36,2021-11-03,6,4,46.0,-0.454545,1.175325,-0.666667,1481547,339.125252,3391.252522,3.104863


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      88.282436
std       65.471651
min       19.362381
25%       45.687187
50%       71.096491
75%      103.941462
max      339.125252
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
20,島根県,21056,1724,5,1778.02,2021-11-03,2,0,42.0,-0.333333,-0.056890,-0.750000,679324,25.378170,253.781701,6.182617
9,大阪府,17766,202562,3047,16087.13,2021-11-03,53,0,303.0,0.472222,0.486859,-0.196970,8849635,228.893056,2288.930560,3.423870
18,岡山県,21714,15332,136,5667.25,2021-11-03,12,0,63.0,-0.142857,0.173784,0.090909,1903627,80.540988,805.409883,3.309472
17,岐阜県,13818,18820,218,6618.65,2021-11-03,18,0,67.0,0.800000,0.490895,0.800000,2032490,92.595782,925.957815,3.296449
28,沖縄県,30926,50243,395,24196.36,2021-11-03,6,4,46.0,-0.454545,1.175325,-0.666667,1481547,339.125252,3391.252522,3.104863
37,群馬県,6580,16804,176,6050.33,2021-11-03,11,0,47.0,1.750000,0.377381,0.100000,1969439,85.323790,853.237902,2.386466
23,愛媛県,25004,5391,82,2812.56,2021-11-03,2,0,30.0,-0.714286,0.572789,-0.800000,1369131,39.375341,393.753410,2.191171
14,山口県,23030,5744,93,2956.52,2021-11-03,0,0,30.0,-1.000000,0.930952,-1.000000,1369882,41.930619,419.306188,2.189970
21,広島県,22372,22083,202,5507.45,2021-11-03,6,0,57.0,-0.250000,0.213955,0.000000,2826858,78.118533,781.185330,2.016373
3,兵庫県,18424,78496,1395,10046.85,2021-11-03,25,0,111.0,1.272727,0.664254,0.388889,5549568,141.445244,1414.452440,2.000156


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    19.000000
mean      0.386265
std       0.350030
min      -0.056890
25%       0.065084
50%       0.377381
75%       0.613736
max       1.175325
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,30926,50243,395,24196.36,2021-11-03,6,4,46.0,-0.454545,1.175325,-0.666667,1481547,339.125252,3391.252522,3.104863
14,山口県,23030,5744,93,2956.52,2021-11-03,0,0,30.0,-1.000000,0.930952,-1.000000,1369882,41.930619,419.306188,2.189970
1,京都府,17108,35892,291,9723.37,2021-11-03,5,0,33.0,0.666667,0.782653,4.000000,2545899,140.979670,1409.796697,1.296202
3,兵庫県,18424,78496,1395,10046.85,2021-11-03,25,0,111.0,1.272727,0.664254,0.388889,5549568,141.445244,1414.452440,2.000156
5,千葉県,7896,100325,1025,11216.77,2021-11-03,13,0,65.0,-0.235294,0.654682,0.857143,6319772,158.747816,1587.478156,1.028518
23,愛媛県,25004,5391,82,2812.56,2021-11-03,2,0,30.0,-0.714286,0.572789,-0.800000,1369131,39.375341,393.753410,2.191171
17,岐阜県,13818,18820,218,6618.65,2021-11-03,18,0,67.0,0.800000,0.490895,0.800000,2032490,92.595782,925.957815,3.296449
9,大阪府,17766,202562,3047,16087.13,2021-11-03,53,0,303.0,0.472222,0.486859,-0.196970,8849635,228.893056,2288.930560,3.423870
4,北海道,658,60779,1471,8099.82,2021-11-03,13,0,62.0,-0.071429,0.485503,0.444444,5267762,115.379169,1153.791686,1.176970
37,群馬県,6580,16804,176,6050.33,2021-11-03,11,0,47.0,1.750000,0.377381,0.100000,1969439,85.323790,853.237902,2.386466


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
20,島根県,21056,1724,5,1778.02,2021-11-03,2,0,42.0,-0.333333,-0.056890,-0.750000,679324,25.378170,253.781701,6.182617
32,神奈川県,9212,168920,1308,12853.55,2021-11-03,7,0,63.0,-0.300000,-0.034184,-0.533333,9209442,183.420450,1834.204504,0.684081
34,福岡県,26320,74447,621,10207.06,2021-11-03,5,0,60.0,0.000000,0.056256,-0.444444,5129841,145.125356,1451.253557,1.169627
26,東京都,8554,381707,3152,19190.76,2021-11-03,25,3,142.0,0.388889,0.062718,-0.305556,13834925,275.901026,2759.010258,1.026388
29,滋賀県,16450,12410,103,6142.01,2021-11-03,0,0,9.0,-1.000000,0.064286,-1.000000,1420948,87.336060,873.360602,0.633380
7,埼玉県,7238,115623,1046,11009.45,2021-11-03,7,0,69.0,0.000000,0.065882,-0.500000,7390054,156.457585,1564.575847,0.933687
24,愛知県,15134,106677,1156,9884.13,2021-11-03,22,0,111.0,1.200000,0.171931,0.100000,7575530,140.817870,1408.178702,1.465244
18,岡山県,21714,15332,136,5667.25,2021-11-03,12,0,63.0,-0.142857,0.173784,0.090909,1903627,80.540988,805.409883,3.309472
21,広島県,22372,22083,202,5507.45,2021-11-03,6,0,57.0,-0.250000,0.213955,0.000000,2826858,78.118533,781.185330,2.016373
37,群馬県,6580,16804,176,6050.33,2021-11-03,11,0,47.0,1.750000,0.377381,0.100000,1969439,85.323790,853.237902,2.386466


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
for i,v in enumerate(chart_data["各地の感染者数_1日ごとの発表数"]):
    plt.text(v,i,"{:,}".format(v),va="center",fontsize=7)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
for i,v in enumerate(chart_data["直近1週間の10万人当たりの感染者数"]):
    plt.text(v,i,"{:,}".format(int(v)),va="center",fontsize=7)
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[37  4  3  1  1  0  0  1]
[  1668.     49172.875  96677.75  144182.625 191687.5   239192.375
 286697.25  334202.125 381707.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([33.,  4.,  4.,  0.,  3.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 1.,  7., 12.,  8.,  7.,  2.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
23020,2021-10-25,35,山口県,1,5711,0,93,1.47,20.0,-0.750000,0.238095,-0.750000
23021,2021-10-26,35,山口県,2,5713,0,93,1.40,19.0,1.000000,0.416667,-0.333333
23022,2021-10-27,35,山口県,1,5714,0,93,1.18,16.0,-0.500000,0.297619,-0.750000
23023,2021-10-28,35,山口県,6,5720,0,93,1.55,21.0,5.000000,1.119048,5.000000
23024,2021-10-29,35,山口県,10,5730,0,93,1.99,27.0,0.666667,0.785714,1.500000
23025,2021-10-30,35,山口県,4,5734,0,93,2.06,28.0,-0.600000,0.735714,0.333333
23026,2021-10-31,35,山口県,5,5739,0,93,2.14,29.0,0.250000,0.723810,0.250000
23027,2021-11-01,35,山口県,1,5740,0,93,2.14,29.0,-0.800000,0.716667,0.000000
23028,2021-11-02,35,山口県,4,5744,0,93,2.28,31.0,3.000000,1.002381,1.000000
23029,2021-11-03,35,山口県,0,5744,0,93,2.21,30.0,-1.000000,0.930952,-1.000000
